1. 이미지분석
2. custom vision- clssification
3. face 
4. OCR
5. Video Indexer

# Azure AI Vision 솔루션

* .NET 패키지 설치

In [31]:
#r "nuget: Microsoft.Extensions.Configuration.Json, 3.1.3"
#r "nuget: System.Drawing.Common, 4.7.0"
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.ComputerVision"
#r "nuget: Azure.AI.Vision.Common"
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.Face, 2.8.0-preview.3"
#r "nuget: Azure.AI.Vision.Core, 0.10.0-beta.1"
#r "nuget: Azure.AI.Vision.ImageAnalysis, 1.0.0-beta.2"
#r "nuget: Microsoft.Azure.CognitiveServices.Language.LUIS.Runtime, 3.0.0"

Installed Packages Azure.AI.Vision.Common, 0.11.0 Azure.AI.Vision.Core, 0.10.0-beta.1 Azure.AI.Vision.ImageAnalysis, 1.0.0-beta.2 Microsoft.Azure.CognitiveServices.Language.LUIS.Runtime, 3.0.0 Microsoft.Azure.CognitiveServices.Vision.ComputerVision, 7.0.1 Microsoft.Azure.CognitiveServices.Vision.Face, 2.8.0-preview.3 Microsoft.Extensions.Configuration.Json, 3.1.3 System.Drawing.Common, 4.7.0

* namespace 가져오기

In [32]:
//필요한 것만 남기기
using System;
using System.IO;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Threading.Tasks;
using System.Drawing;
using Microsoft.Extensions.Configuration;
using Azure;
using Azure.AI.Vision.ImageAnalysis;
using Azure.AI.Vision.Common;
using Azure.AI.Vision;
using Microsoft.Azure.CognitiveServices.Vision.Face;
using Microsoft.Azure.CognitiveServices.Vision.Face.Models;
using Microsoft.Azure.CognitiveServices.Vision.ComputerVision;
using Microsoft.Azure.CognitiveServices.Vision.ComputerVision.Models;
using Azure.AI.Vision.Core.Options;
using Azure.AI.Vision.Core.Input;
using System.Linq;
using System.Collections.Generic;
using System.Threading;
using Microsoft.Azure.CognitiveServices.Language.LUIS.Runtime;
using Microsoft.Rest;


# 1. 이미지 분석

In [33]:
// 사용할 이미지의 주소 가져오기
string address_1="C:\\Users\\parkm\\Desktop\\Vision\\mslearn-ai-vision\\Labfiles\\01-analyze-images\\C-Sharp\\image-analysis\\";
public string imageFile = address_1+"images/street.jpg";

In [34]:
//API 사용을 위한 인증 절차
public string aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
public string aiSvcKey ="68309624561e49688c4cbc6143304744";

public var client = new ImageAnalysisClient(new Uri(aiSvcEndpoint), new AzureKeyCredential(aiSvcKey));

In [35]:
//이미지 분석 API 처리 함수
public static void AnalyzeImage(string imageFile, ImageAnalysisClient client)
        {
            Console.WriteLine($"\nAnalyzing {imageFile} \n");

            // 이미지 파일을 전달할 수 있도록 파일 스트림 형태로 바꾸기
            using FileStream stream = new FileStream(imageFile,
                                                     FileMode.Open);

            // 이미지 분석 결과 가져오기
            ImageAnalysisResult result = client.Analyze(
                BinaryData.FromStream(stream),
                VisualFeatures.Caption | VisualFeatures.DenseCaptions | VisualFeatures.Objects | VisualFeatures.Tags | VisualFeatures.People);
                        
            
            // 가장 정확한 이미지 분석 결과 출력
            if (result.Caption.Text != null)
            {
                Console.WriteLine(" Caption:");
                Console.WriteLine($"   \"{result.Caption.Text}\", 신뢰도 {result.Caption.Confidence:0.00}\n");
            }

            // 이미지에 대한 다양한 분석 결과 출력
            Console.WriteLine(" Dense Captions:");
            foreach (DenseCaption denseCaption in result.DenseCaptions.Values)
            {
                Console.WriteLine($"   Caption: '{denseCaption.Text}', 신뢰도: {denseCaption.Confidence:0.00}");
            }

        }

In [36]:
// 이미지 분석
AnalyzeImage(imageFile, client);


Analyzing C:\Users\parkm\Desktop\Vision\mslearn-ai-vision\Labfiles\01-analyze-images\C-Sharp\image-analysis\images/street.jpg 

 Caption:
   "a man walking a dog on a leash on a street", 신뢰도 0.82

 Dense Captions:
   Caption: 'a man walking a dog on a leash on a street', 신뢰도: 0.82
   Caption: 'a man walking on a street', 신뢰도: 0.69
   Caption: 'a yellow car on the street', 신뢰도: 0.78
   Caption: 'a black dog walking on the street', 신뢰도: 0.75
   Caption: 'a blurry image of a blue car', 신뢰도: 0.82
   Caption: 'a yellow taxi cab on the street', 신뢰도: 0.72


In [37]:
 // 이미지 배경화면 제거 API 처리 함수
public static async Task BackgroundForeground(string imageFile, string endpoint, string key){
           
            Console.WriteLine($" Background removal:");
            // API 버전과 모드 지정
            string apiVersion = "2023-02-01-preview";
            string mode = "backgroundRemoval"; // "foregroundMatting" 또는 "backgroundRemoval" 지정 가능
            string url = $"computervision/imageanalysis:segment?api-version={apiVersion}&mode={mode}";

            // API 호출
            using (var client = new HttpClient())
            {
                var contentType = new MediaTypeWithQualityHeaderValue("application/json");
                client.BaseAddress = new Uri(endpoint);
                client.DefaultRequestHeaders.Accept.Add(contentType);
                client.DefaultRequestHeaders.Add("Ocp-Apim-Subscription-Key", key);

                var data = new{
                    url = $"https://github.com/MicrosoftLearning/mslearn-ai-vision/blob/main/Labfiles/01-analyze-images/Python/image-analysis/images/street.jpg?raw=true"
                };

                var jsonData = JsonSerializer.Serialize(data);
                var contentData = new StringContent(jsonData, Encoding.UTF8, contentType);
                var response = await client.PostAsync(url, contentData);

                File.WriteAllBytes("background.png", response.Content.ReadAsByteArrayAsync().Result);
                Console.WriteLine("  Results saved in background.png\n");
                
            }
            
}

In [38]:
// 배경 제거
await BackgroundForeground(imageFile, aiSvcEndpoint, aiSvcKey);

 Background removal:
  Results saved in background.png



# 3.얼굴 감지

* Computer Vision (얼굴 이정표)

In [39]:
// 사용할 이미지의 주소 가져오기
public string imageFile = "C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/computer-vision/images/people.jpg";

In [40]:
//API 사용을 위한 인증 절차
var aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
var aiSvcKey = "3446cc4f896148e79b4864c0d86fd497";

public var client = new ImageAnalysisClient(
    new Uri(aiSvcEndpoint),
    new AzureKeyCredential(aiSvcKey)
);

In [41]:
//얼굴 이정표 분석 API 처리 함수
public static void AnalyzeImage(string imageFile, ImageAnalysisClient client)
{
    Console.WriteLine($"\nAnalyzing {imageFile} \n");

    // 이미지 파일을 전달할 수 있도록 파일 스트림 형태로 바꾸기
    using FileStream stream = new FileStream(imageFile, FileMode.Open);

    // 이미지에 있는 사람 감지 결과 가져오기
    ImageAnalysisResult result = client.Analyze(
        BinaryData.FromStream(stream),
        VisualFeatures.People);

    // 감지 결과 출력하기
    Console.WriteLine($"이미지에서의 얼굴 분석 결과:");
    Console.WriteLine($" Metadata: Model: {result.ModelVersion} Image dimensions: {result.Metadata.Width} x {result.Metadata.Height}");
    Console.WriteLine($" 분석된 사람들:");
    foreach (DetectedPerson person in result.People.Values)
    {
        if(person.Confidence > 0.3)
            Console.WriteLine($"   사람 : Bounding box {person.BoundingBox.ToString()}, 정확도 {person.Confidence:F4}");
    }
}

In [42]:
//얼굴 이정표 분석
AnalyzeImage(imageFile, client);


Analyzing C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/computer-vision/images/people.jpg 

이미지에서의 얼굴 분석 결과:
 Metadata: Model: 2023-10-01 Image dimensions: 800 x 514
 분석된 사람들:
   사람 : Bounding box Top: 73, Left: 454, Width: 345, Height: 439, 정확도 0.9518
   사람 : Bounding box Top: 0, Left: 87, Width: 491, Height: 513, 정확도 0.9195


* Face API (얼굴 특징 분석)

In [43]:
// 사용할 이미지의 주소 가져오기
public string imageFile = "C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/face-api/images/people.jpg";

In [44]:
//API 사용을 위한 인증 절차
string cogSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
string cogSvcKey = "3446cc4f896148e79b4864c0d86fd497";

Microsoft.Azure.CognitiveServices.Vision.Face.ApiKeyServiceClientCredentials credentials = new Microsoft.Azure.CognitiveServices.Vision.Face.ApiKeyServiceClientCredentials(cogSvcKey);

public FaceClient client = new FaceClient(credentials)
{
    Endpoint = cogSvcEndpoint
};

In [45]:
// API에게 요청할 내용 설정
public static IList<FaceAttributeType> SetFaceFeatures()
{
    IList<FaceAttributeType> features = new FaceAttributeType[]
    {
        FaceAttributeType.Occlusion, //얼굴 부분 가리는 정도
        FaceAttributeType.Blur, //흐림 정도
        FaceAttributeType.Glasses //안경 착용 여부
    };

    return features;
}

In [46]:
// 이미지를 파일 스트림 형태로 변환하는 함수
public static FileStream ReadImageData(string imageFile)
{
    var imageData = File.OpenRead(imageFile);
    return imageData;
}

In [47]:
// 얼굴 특징 분석 API 처리 함수
public static async Task<List<DetectedFace>> DetectFaces(string imageFile, FaceClient client)
{
    Console.WriteLine($"Detecting faces in {imageFile}");

    IList<FaceAttributeType> features = SetFaceFeatures(); // 앞서 지정한 얼굴 특징들 API에게 요청

    using (var imageData = ReadImageData(imageFile))
    {
        var detected_faces = await client.Face.DetectWithStreamAsync(imageData, returnFaceAttributes: features, returnFaceId: false);
        return detected_faces.ToList();
    }
}

In [50]:
//얼굴 특징 분석 !!!!!!!!!!!!!!!!!!!!!!-> 디텍팅하게 수정
await DetectFaces(imageFile, client);

Detecting faces in C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/face-api/images/people.jpg


# 4. OCR

In [5]:
// 사용할 이미지의 주소 가져오기
 public static String address_3="C:\\Users\\parkm\\Desktop\\Vision\\mslearn-ai-vision\\Labfiles\\05-ocr\\C-Sharp\\read-text\\images\\";

In [22]:
//API 사용을 위한 인증 절차
public string aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
public string aiSvcKey = "68309624561e49688c4cbc6143304744";


public ImageAnalysisClient client = new ImageAnalysisClient(
    new Uri(aiSvcEndpoint),
    new AzureKeyCredential(aiSvcKey)
);


In [25]:
// OCR API 처리 함수
public static void GetTextRead(string imageFile, ImageAnalysisClient client){
    Console.WriteLine($"\nReading text from {imageFile} \n");

    // 이미지를 파일 스트림 형태로 변환
    using FileStream stream = new FileStream(imageFile,
                                                     FileMode.Open);

    // 이미지 분석 결과 받기
    ImageAnalysisResult result = client.Analyze(
    BinaryData.FromStream(stream),
    VisualFeatures.Read);
                
    stream.Close();
                
    // 분석 결과 보여주기
    if (result.Read != null){
        Console.WriteLine($"Text:");
                
        
        System.Drawing.Image image = System.Drawing.Image.FromFile(imageFile);
        Graphics graphics = Graphics.FromImage(image);
        Pen pen = new Pen(Color.Cyan, 3);
                    
        foreach (var line in result.Read.Blocks.SelectMany(block => block.Lines)){
            // 분석 결과 출력
            Console.WriteLine($"   '{line.Text}'");
                        
            var drawLinePolygon = true;
                        
            Console.WriteLine($"   Bounding Polygon: [{string.Join(" ", line.BoundingPolygon)}]");   
                        
            foreach (DetectedTextWord word in line.Words){
                Console.WriteLine($"     Word: '{word.Text}', Confidence {word.Confidence:F4}, Bounding Polygon: [{string.Join(" ", word.BoundingPolygon)}]");
                            
                drawLinePolygon = false;
                var r = word.BoundingPolygon;
                        
                Point[] polygonPoints = {
                    new Point(r[0].X, r[0].Y),
                    new Point(r[1].X, r[1].Y),
                    new Point(r[2].X, r[2].Y),
                    new Point(r[3].X, r[3].Y)
                };
                        
                graphics.DrawPolygon(pen, polygonPoints);
            }
                        
                        
            // 이미지 위에 분석 결과 나타내기
            if (drawLinePolygon) {
                var r = line.BoundingPolygon;
                        
                Point[] polygonPoints = {
                    new Point(r[0].X, r[0].Y),
                    new Point(r[1].X, r[1].Y),
                    new Point(r[2].X, r[2].Y),
                    new Point(r[3].X, r[3].Y)
                };
                        
            graphics.DrawPolygon(pen, polygonPoints);
            }
        }
                        
    // 분석 내용이 반영된 이미지 저장
    String output_file = "text.jpg";
    image.Save(output_file);
    Console.WriteLine("\nResults saved in " + output_file + "\n");   
    }
}

In [26]:
//1. Lincoln.jpg 읽기
GetTextRead(address_3+"Lincoln.jpg", client);


Reading text from C:\Users\parkm\Desktop\Vision\mslearn-ai-vision\Labfiles\05-ocr\C-Sharp\read-text\images\Lincoln.jpg 

Text:
   'IN THIS TEMPLE'
   Bounding Polygon: [(328, 171) (477, 169) (477, 184) (328, 186)]
     Word: 'IN', Confidence 0.9930, Bounding Polygon: [(328, 171) (342, 171) (342, 187) (328, 187)]
     Word: 'THIS', Confidence 0.9910, Bounding Polygon: [(357, 171) (397, 170) (397, 185) (357, 186)]
     Word: 'TEMPLE', Confidence 0.9930, Bounding Polygon: [(407, 170) (474, 170) (474, 184) (407, 185)]
   'AS IN THE HEARTS OF THE PEOPLE'
   Bounding Polygon: [(240, 193) (564, 188) (564, 203) (240, 210)]
     Word: 'AS', Confidence 0.9950, Bounding Polygon: [(241, 194) (262, 194) (262, 210) (241, 211)]
     Word: 'IN', Confidence 0.9980, Bounding Polygon: [(270, 193) (284, 193) (284, 210) (270, 210)]
     Word: 'THE', Confidence 0.9930, Bounding Polygon: [(298, 193) (332, 192) (332, 208) (298, 209)]
     Word: 'HEARTS', Confidence 0.9940, Bounding Polygon: [(340, 192) (410,

In [ ]:
//2.Note.jpg 읽기
GetTextRead(address_3+"Note.jpg", client);